In [1]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
from collections import Counter
from nltk.tokenize import LineTokenizer, SpaceTokenizer, TweetTokenizer
from nltk import word_tokenize
from nltk import word_tokenize, PorterStemmer, WordNetLemmatizer
import string
import matplotlib
import re
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from sklearn.linear_model import Ridge
import numpy as np
from sklearn.metrics import mean_squared_error
from scipy.sparse import csr_matrix, hstack


In [2]:
df_miss_50000=pd.read_pickle('/Users/vaibhavagarwal/Desktop/df_50000')

In [3]:
df_miss_50000.head(5)

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,sub_cat1,sub_cat2,sub_cat3,text,tokens,Size,MostCommonWord,brand_prediction,confidence_level,brand_name_predicted
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,MLB,10.0,1,No description yet,Men,Tops,T-shirts,MLB Cincinnati Reds T Shirt Size XL Men Tops T...,"[mlb, cincinnati, red, shirt, size, xl, men, t...",[XL],mlb,"[(1, MLB)]",1.000000,MLB
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,iHome,35.0,1,New with tags. Leather horses. Retail for [rm]...,Home,Home Décor,Home Décor Accents,Leather Horse Statues Home Home Décor Home Déc...,"[leather, horse, statue, home, home, décor, ho...",[],home,"[(0.8, iHome)]",0.800000,iHome
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity,Women,Jewelry,Necklaces,24K GOLD plated rose Women Jewelry Necklaces n...,"[24k, gold, plated, rose, woman, jewelry, neck...",[],24k,"[(0.09999999999999998, 24/7 Comfort Apparel)]",0.100000,24/7 Comfort Apparel
5,5,Bundled items requested for Ruie,3,Women/Other/Other,TomTom,59.0,0,"Banana republic bottoms, Candies skirt with ma...",Women,Other,Other,Bundled items requested for Ruie Women Other O...,"[bundled, item, requested, ruie, woman, nan, b...",[],bottom,"[(0.6666666666666667, TomTom), (0.666666666666...",0.666667,TomTom
9,9,Porcelain clown doll checker pants VTG,3,Vintage & Collectibles/Collectibles/Doll,Lowa,8.0,0,I realized his pants are on backwards after th...,Vintage & Collectibles,Collectibles,Doll,Porcelain clown doll checker pants VTG Vintage...,"[porcelain, clown, doll, checker, pant, vtg, v...",[],clown,"[(0.6, Lowa), (0.6, CLEAN)]",0.600000,Lowa


In [4]:
def checkforfloat(word):
    if re.search('[0-9]', word):
        return (word)

df_miss_50000['cat_with_float']=df_miss_50000['sub_cat3'].map(checkforfloat)        
            
# getting the mostcommon word used for each row in the dataset
# def getmostCommonword(tokens):
#     fdistPlain = nltk.FreqDist(tokens)
#     return (fdistPlain.most_common(1)[0][0])

# df_miss_50000['MostCommonWord']=df_miss_50000['tokens'].map(getmostCommonword)
# df_miss_50000.head(100)


In [5]:
df_miss_50000[~(df_miss_50000.cat_with_float.isna())]

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,sub_cat1,sub_cat2,sub_cat3,text,tokens,Size,MostCommonWord,brand_prediction,confidence_level,brand_name_predicted,cat_with_float
1449,1449,Indian cents,3,Vintage & Collectibles/Antique/100 Years or Older,Indah,14.0,0,1902.1903.1905.1907.,Vintage & Collectibles,Antique,100 Years or Older,Indian cents Vintage & Collectibles Antique 10...,"[indian, cent, vintage, collectible, antique, ...",[],indian,"[(0.6666666666666667, Indah)]",0.666667,Indah,100 Years or Older
2435,2435,VINTAGE!(4)1940'S ASCO SPICE TINS,3,Vintage & Collectibles/Antique/50 To 75 Years,Justin,20.0,0,These are in fair too good condition with some...,Vintage & Collectibles,Antique,50 To 75 Years,VINTAGE!(4)1940'S ASCO SPICE TINS Vintage & Co...,"[vintage, asco, spice, tin, vintage, collectib...",[],tin,"[(0.5, Justin), (0.5, Tide), (0.5, Tilt), (0.5...",0.500000,Justin,50 To 75 Years
2948,2948,U.S. 1969 P & D Coin Set In Cellophane,2,Vintage & Collectibles/Antique/50 To 75 Years,Com,10.0,1,Up for sale: *1 U.S. 1969 P & D Coin Set In Ce...,Vintage & Collectibles,Antique,50 To 75 Years,U.S. 1969 P & D Coin Set In Cellophane Vintage...,"[u, p, coin, set, cellophane, vintage, collect...",[],coin,"[(0.5, Com), (0.5, Cos), (0.5, Coty), (0.5, GA...",0.500000,Com,50 To 75 Years
4763,4763,Customcoinrings 5 quarters silver,4,Vintage & Collectibles/Antique/100 Years or Older,Custom Accessories,23.0,1,Coin collection,Vintage & Collectibles,Antique,100 Years or Older,Customcoinrings 5 quarters silver Vintage & Co...,"[customcoinrings, quarter, silver, vintage, co...",[],customcoinrings,"[(0.5, Custom Accessories)]",0.500000,Custom Accessories,100 Years or Older
4982,4982,Antique Iron,3,Vintage & Collectibles/Antique/75 To 100 Years,IRO,21.0,0,"American Beauty Iron. Wood handle. Iron, stand...",Vintage & Collectibles,Antique,75 To 100 Years,Antique Iron Vintage & Collectibles Antique 75...,"[antique, iron, vintage, collectible, antique,...",[],iron,"[(0.75, IRO), (0.75, Icon)]",0.750000,IRO,75 To 100 Years
7359,7359,Antique enamelware pale bucket wooden ha,4,Vintage & Collectibles/Antique/50 To 75 Years,Latique,27.0,0,Vintage antique enamelware bucket with wooden ...,Vintage & Collectibles,Antique,50 To 75 Years,Antique enamelware pale bucket wooden ha Vinta...,"[antique, enamelware, pale, bucket, wooden, ha...",[],antique,"[(0.7142857142857143, Latique), (0.71428571428...",0.714286,Latique,50 To 75 Years
8864,8864,Old coins,3,Vintage & Collectibles/Antique/100 Years or Older,Olay,7.0,0,U.S .MINT.philadelphia 1969 dept of the,Vintage & Collectibles,Antique,100 Years or Older,Old coins Vintage & Collectibles Antique 100 Y...,"[old, coin, vintage, collectible, antique, yea...",[],old,"[(0.5, Olay), (0.5, Olga), (0.5, Holden)]",0.500000,Olay,100 Years or Older
29741,29741,1858 Flying Eagle Cent,4,Vintage & Collectibles/Antique/100 Years or Older,Com,8.0,1,1858 Flying Eagle Cent in great shape for its ...,Vintage & Collectibles,Antique,100 Years or Older,1858 Flying Eagle Cent Vintage & Collectibles ...,"[flying, eagle, cent, vintage, collectible, an...",[],coin,"[(0.5, Com), (0.5, Cos), (0.5, Coty), (0.5, GA...",0.500000,Com,100 Years or Older
36913,36913,U.S. Series 1935 E Silver Certificate,3,Vintage & Collectibles/Antique/75 To 100 Years,NaN,7.0,1,Up for sale: *1 U.S. Series 1935 E [rm] Silver...,Vintage & Collectibles,Antique,75 To 100 Years,U.S. Series 1935 E Silver Certificate Vintage ...,"[u, series, e, silver, certificate, vintage, c...",[],certificate,"[(0.4545454545454546, CeraVe), (0.454545454545...",0.454545,CeraVe,75 To 100 Years
49905,49905,Old silver coins,3,Vintage & Collectibles/Antique/100 Years or Older,Olay,24.0,0,1902.and 1914 s barber half dollar,Vintage & Collectibles,Antique,100 Years or Older,Old silver coins Vintage & Collectibles Antiqu...,"[old, silver, coin, vintage, collectible, anti...",[],old,"[(0.5, Olay), (0.5, Olga), (0.5, Holden)]",0.500000,Olay,100 Years 

## Ridge implementation

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
    

X_cat1 = vectorizer.fit_transform(df_miss_50000['sub_cat1'])
X_cat2 = vectorizer.fit_transform(df_miss_50000['sub_cat2'])
X_cat3 = vectorizer.fit_transform(df_miss_50000['sub_cat3'])
X_item = vectorizer.fit_transform(df_miss_50000['item_description'])
X_most_common_word=vectorizer.fit_transform(df_miss_50000['MostCommonWord'])
X_dummies = csr_matrix(pd.get_dummies(df_miss_50000[['item_condition_id', 'shipping']],sparse=True).values)



In [7]:
print(vectorizer.get_feature_names())

['00g', '10c', '10ft', '10k', '10lbs', '10pc', '10x13', '12m', '12x15', '14g', '14k', '14kt', '14x17', '1576ad', '15ml', '15oz', '16gb', '16oz', '18g', '18k', '18kgf', '18m', '1914p', '1964p', '1966p', '1d', '1oz', '1pc', '1pk', '1pr', '1st', '1wunderbrow', '1x', '2003s', '200ct', '20oz', '20yds', '21mm', '21oz', '21st', '22k', '24hr', '24k', '24month', '24pc', '25ft', '2g', '2letters', '2ml', '2ndtimearound', '2oz', '2pcs', '2size8', '2t', '2x', '2xl', '2y', '300ft', '30a', '30ml', '30th', '30x', '32cm', '32x32', '33h', '34a', '36a', '36c', '36d', '38b', '38c', '38mm', '3c', '3d', '3g', '3m', '3oz', '3pair', '3pc', '3pcs', '3pr', '3t', '3x', '3x5', '3xl', '3y', '40xp', '42mm', '44d', '49ers', '4c', '4gb', '4k', '4life', '4ml', '4mm', '4oz', '4pcs', '4t', '4th', '4x', '4x8', '4xl', '50ft', '5c', '5ft', '5lanyards10stickers', '5mm', '5oz', '5packs', '5pairs', '5pc', '5t', '5th', '5x7', '5xl', '5y', '64gb', '6c', '6ft', '6mm', '6pcs', '6x', '6x9', '6y', '7x', '7x10', '805princess', '8g',

In [8]:
print(X_dummies.toarray())

[[3 1]
 [1 1]
 [1 0]
 ...
 [1 1]
 [1 1]
 [2 1]]


In [9]:
y=df_miss_50000['price'].values.reshape(-1,1)
y

array([[10.],
       [35.],
       [44.],
       ...,
       [ 8.],
       [53.],
       [ 8.]])

In [10]:
from scipy.sparse import csr_matrix, hstack
sparse_merge = hstack((X_cat1, X_cat2,X_cat3,X_item,X_most_common_word,X_dummies)).tocsr()
sparse_merge.shape

(50000, 40832)

In [11]:
X_train=sparse_merge[:30000]
X_test = sparse_merge[30000:]
y_train=y[0:30000]
y_test=y[30000:]
    
model = Ridge(alpha=.05, copy_X=True, fit_intercept=True, max_iter=100,normalize=False, random_state=101, solver='auto', tol=0.001)
model.fit(X_train, y_train)
print("Train ridge completed")


/Users/vaibhavagarwal/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Ridge(alpha=0.05, copy_X=True, fit_intercept=True, max_iter=100,
   normalize=False, random_state=101, solver='auto', tol=0.001)

Train ridge completed


In [12]:
predsR = model.predict(X=X_test)

In [13]:
type(predsR)
predsR.reshape(-1,1).size
y_test.size
predsR

numpy.ndarray

20000

20000

array([[16.47678207],
       [ 9.85500917],
       [ 7.1071711 ],
       ...,
       [15.62734485],
       [28.91483858],
       [ 2.37828207]])

In [14]:
model.score(X_train,y_train) # it means this is used on the training data only

0.5026481653357955

In [15]:
y_pred_train=model.predict(X_train)
rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
print("Root Mean Squared Error: {}".format(rmse))

Root Mean Squared Error: 17.389180655843482


## Random Forest Regressor

In [16]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators=100, max_depth=10, min_samples_split=3)
regressor.fit(X_train, y_train)
predictions = regressor.predict(X_test)
#print(predictions)

/Users/vaibhavagarwal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=3,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [17]:
predictions

array([23.61200093, 23.47862792, 16.48684167, ..., 16.48684167,
       16.48684167, 16.48684167])

In [18]:
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 13.87 degrees.


In [19]:
y_test[0]
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print("Root Mean Squared Error: {}".format(rmse))


array([8.])

Root Mean Squared Error: 26.028410602760054


## Support Vector Regression

In [20]:
from sklearn.svm import SVR
regressor = SVR(C=0.1, epsilon=0.02, kernel='linear')
regressor.fit(X_train, y_train.reshape(-1,1))
predictions = regressor.predict(X_test)
print(predictions)

/Users/vaibhavagarwal/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=0.1, cache_size=200, coef0=0.0, degree=3, epsilon=0.02, gamma='auto',
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

[10.77235096 12.28237341  6.76605041 ... 10.71451097 18.26284274
  8.59482335]


In [21]:
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 13.46 degrees.


## Stochastic Gradient Descent (SGD Regressor)

In [22]:
#MSE= mean squared error
# RMSE= root mean suqared error
# MAE= Mean absolute error
# R2= represents goodness of fit to model

from sklearn.linear_model import SGDRegressor
params = {  "alpha": [0.000007, 0.000006, 0.000005], "penalty": [None, "l2"],  "eta0": [0.001, 0.005, 0.01], "n_iter": [300, 1000, 3000]
}
from sklearn.model_selection import GridSearchCV
regressor = SGDRegressor(loss='squared_loss',penalty='l2', alpha=0.0001, learning_rate='constant', eta0=0.01, n_iter=1000) 
grid_search = GridSearchCV(regressor, params, cv=3)  

grid_search.fit(X_train, y_train)
print(grid_search.best_params_)

regressor_best = grid_search.best_estimator_  

# predict the testing set with the optimal model:

predictions = regressor_best.predict(X_test)  

#evaluate the performance on testing sets based on metrics MSE, MAE, and :

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
mean_squared_error(y_test, predictions)

mean_absolute_error(y_test, predictions)

r2_score(y_test, predictions)


/Users/vaibhavagarwal/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/Users/vaibhavagarwal/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vaibhavagarwal/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/Users/vaibhavagarwal/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, )

/Users/vaibhavagarwal/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/Users/vaibhavagarwal/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vaibhavagarwal/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/Users/vaibhavagarwal/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, )

/Users/vaibhavagarwal/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/Users/vaibhavagarwal/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vaibhavagarwal/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/Users/vaibhavagarwal/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, )

/Users/vaibhavagarwal/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/Users/vaibhavagarwal/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vaibhavagarwal/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/Users/vaibhavagarwal/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, )

/Users/vaibhavagarwal/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/Users/vaibhavagarwal/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vaibhavagarwal/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/Users/vaibhavagarwal/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, )

...

GridSearchCV(cv=3, error_score='raise',
       estimator=SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='constant',
       loss='squared_loss', max_iter=None, n_iter=1000, penalty='l2',
       power_t=0.25, random_state=None, shuffle=True, tol=None, verbose=0,
       warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'alpha': [7e-06, 6e-06, 5e-06], 'penalty': [None, 'l2'], 'eta0': [0.001, 0.005, 0.01], 'n_iter': [300, 1000, 3000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

{'alpha': 5e-06, 'eta0': 0.001, 'n_iter': 3000, 'penalty': None}


1.2906745688154538e+20

799742194.2420195

-1.833134006190007e+17

In [23]:
## feature engineering logic
#  K = [10, 15, 25, 35, 45]
# >>> for k in K:
# ...     top_K_features = feature_sorted[-k:]
# ...     X_k_selected = X[:, top_K_features]
# ...     # Estimate accuracy on the data set with k selected 
#           features
# ...     classifier = SVC(gamma=0.005)
# ...     score_k_features = 
#                cross_val_score(classifier, X_k_selected, y).mean()
# ...     print('Score with the data set of top {0} features: 
#                               {1:.2f}'.format(k, score_k_features))
# ...
# Score with the data set of top 10 features: 0.88
# Score with the data set of top 15 features: 0.93
# Score with the data set of top 25 features: 0.94
# Score with the data set of top 35 features: 0.92
# Score with the data set of top 45 features: 0.88  

In [24]:
## >>> from sklearn.decomposition import PCA
# >>> # Keep different number of top components
# >>> N = [10, 15, 25, 35, 45]
# >>> for n in N:
# ...     pca = PCA(n_components=n)
# ...     X_n_kept = pca.fit_transform(X)
# ...     # Estimate accuracy on the data set with top n components
# ...     classifier = SVC(gamma=0.005)
# ...     score_n_components = 
#                    cross_val_score(classifier, X_n_kept, y).mean()
# ...     print('Score with the data set of top {0} components: 
#                             {1:.2f}'.format(n, score_n_components))
# Score with the data set of top 10 components: 0.95
# Score with the data set of top 15 components: 0.95
# Score with the data set of top 25 components: 0.91
# Score with the data set of top 35 components: 0.89
# Score with the data set of top 45 components: 0.88  

In [25]:
# >>> from sklearn.preprocessing import StandardScaler 
# >>> scaler = StandardScaler()
# >>> scaler.fit(X_train)
# Now save the established standardize, the scaler object with pickle:

# >>> import pickle
# >>> pickle.dump(scaler, open("scaler.p", "wb" ))
# This generates the scaler.p file. Move on with training a SVR model on the scaled data:

# >>> X_scaled_train = scaler.transform(X_train) 
# >>> from sklearn.svm import SVR 
# >>> regressor = SVR(C=20) 
# >>> regressor.fit(X_scaled_train, y_train)
# Save the trained regressor, the regressor object with pickle:

# >>> pickle.dump(regressor, open("regressor.p", "wb"))
# This generates the regressor.p file. In the deployment stage, we first load in the saved standardizer and regressor from the two preceding files:

# >>> my_scaler = pickle.load(open("scaler.p", "rb" )) 
# >>> my_regressor = pickle.load(open("regressor.p", "rb"))

In [26]:
# ##
# Naive Bayes and tree-based algorithms can directly work with categorical features, while other algorithms in general cannot, in which case encoding is essential.

In [27]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2)
X_new = poly.fit_transform(X_train.toarray())
print(X_new)

MemoryError: 

In [ ]:
# Python machine learning by example. 